In [1]:
import tensorflow as tf
import tensorflow_data_validation as tfdv
import pandas as pd

from sklearn.model_selection import train_test_split
# from util import add_extra_rows

from tensorflow_metadata.proto.v0 import schema_pb2


import tensorflow_transform as tft
import tensorflow_transform.beam as tft_beam

from tensorflow_transform.tf_metadata import dataset_metadata
from tensorflow_transform.tf_metadata import schema_utils

import pprint
import tempfile

print('TFDV Version: {}'.format(tfdv.__version__))
print('Tensorflow Version: {}'.format(tf.__version__))
print('TFT Version: {}'.format(tft.__version__))

2024-07-05 08:09:40.426542: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-05 08:09:41.030348: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-05 08:09:41.030459: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-05 08:09:41.141733: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-05 08:09:41.354862: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-05 08:09:41.359647: I tensorflow/core/platform/cpu_feature_guard.cc:1

TFDV Version: 1.15.1
Tensorflow Version: 2.15.1
TFT Version: 1.15.0


In [2]:
df = pd.read_csv("/home/kaung_nyo_lwin/AI_Projects/Data/customer_churn_prediction_dataset.csv")
df.pop('customerID')

train_df, eval_df = train_test_split(df, test_size=0.2, shuffle=True)

In [3]:
print("Training Data Set")
display(train_df.head())

Training Data Set


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
97,Female,1,No,No,5,Yes,No,DSL,Yes,No internet service,No,Yes,Yes,No internet service,Month-to-month,Yes,Electronic check,34.37,171.85,Yes
55,Female,0,Yes,Yes,8,Yes,No phone service,Fiber optic,No,Yes,No,No,No,No internet service,Month-to-month,No,Credit card,95.08,760.64,No
238,Male,0,No,No,31,Yes,No phone service,No,Yes,No,Yes,Yes,No,Yes,One year,Yes,Bank transfer,60.95,1889.45,No
281,Female,0,No,No,52,No,No phone service,DSL,No,Yes,No,Yes,No internet service,No internet service,Two year,No,Bank transfer,108.80,5657.60,No
114,Male,0,No,Yes,50,Yes,No,No,Yes,No internet service,Yes,No,No,No internet service,One year,Yes,Mailed check,53.85,2692.50,No


In [4]:
print("Evaluation Data Set")
display(eval_df.head())

Evaluation Data Set


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
224,Female,1,No,No,6,Yes,No,Fiber optic,No internet service,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,94.50,567.00,Yes
213,Male,1,No,Yes,2,No,Yes,No,Yes,No,No,No internet service,Yes,No,Two year,No,Electronic check,98.95,197.90,No
38,Male,1,No,No,38,Yes,Yes,DSL,No internet service,Yes,No internet service,No internet service,No,Yes,Month-to-month,No,Bank transfer,43.21,1641.98,Yes
262,Male,0,No,No,52,No,Yes,Fiber optic,No internet service,Yes,Yes,No,Yes,No internet service,Two year,Yes,Mailed check,96.44,5014.88,Yes
115,Female,0,No,No,10,No,Yes,DSL,Yes,No internet service,No internet service,Yes,No,No internet service,One year,Yes,Mailed check,28.84,288.40,No


In [5]:
# Generate training dataset statistics
train_stats = tfdv.generate_statistics_from_dataframe(train_df)

In [6]:
# Visualize training dataset statistics
tfdv.visualize_statistics(train_stats)

In [7]:
# Infer schema from the computed statistics.
schema = tfdv.infer_schema(statistics=train_stats)

# Display the inferred schema
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'gender',STRING,required,,'gender'
'SeniorCitizen',INT,required,,-
'Partner',STRING,required,,'Partner'
'Dependents',STRING,required,,'Dependents'
'tenure',INT,required,,-
'PhoneService',STRING,required,,'PhoneService'
'MultipleLines',STRING,required,,'MultipleLines'
'InternetService',STRING,required,,'InternetService'
'OnlineSecurity',STRING,required,,'OnlineSecurity'


,Values
Domain,
'gender',"'Female', 'Male'"
'Partner',"'No', 'Yes'"
'Dependents',"'No', 'Yes'"
'PhoneService',"'No', 'Yes'"
'MultipleLines',"'No', 'No phone service', 'Yes'"
'InternetService',"'DSL', 'Fiber optic', 'No'"
'OnlineSecurity',"'No', 'No internet service', 'Yes'"
'OnlineBackup',"'No', 'No internet service', 'Yes'"
'DeviceProtection',"'No', 'No internet service', 'Yes'"


In [8]:
# Generate evaluation dataset statistics
eval_stats = tfdv.generate_statistics_from_dataframe(eval_df)

# Compare training with evaluation
tfdv.visualize_statistics(
    lhs_statistics=eval_stats, 
    rhs_statistics=train_stats, 
    lhs_name='EVAL_DATASET', 
    rhs_name='TRAIN_DATASET'
)

In [9]:
# Check evaluation data for errors by validating the evaluation dataset statistics using the reference schema
anomalies =  tfdv.validate_statistics(statistics=eval_stats, schema=schema)

# Visualize anomalies
tfdv.display_anomalies(anomalies)

In [10]:
CATEGORICAL_FEATURE_KEYS = [
    'gender', 
    'SeniorCitizen', 
    'Partner', 
    'Dependents', 
    'PhoneService', 
    'MultipleLines', 
    'InternetService',
    'OnlineSecurity',
    'OnlineBackup',
    'DeviceProtection',
    'TechSupport',
    'StreamingTV',
    'StreamingMovies',
    'Contract',
    'PaperlessBilling',
    'PaymentMethod'
]

NUMERIC_FEATURE_KEYS = [
    'tenure',
    'MonthlyCharges',
    'TotalCharges'
]


LABEL_KEY = 'Churn'

def transformed_name(key):
    return key + '_xf'

In [11]:
RAW_DATA_FEATURE_SPEC = dict(
    [(name, tf.io.FixedLenFeature([], tf.string))
     for name in CATEGORICAL_FEATURE_KEYS] +
    [(name, tf.io.FixedLenFeature([], tf.float32))
     for name in NUMERIC_FEATURE_KEYS] + 
    [(LABEL_KEY, tf.io.FixedLenFeature([], tf.string))]
)

raw_data_metadata = tft.DatasetMetadata.from_feature_spec(RAW_DATA_FEATURE_SPEC)

In [12]:
# Define the transformations
def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
        inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
        Map from string feature key to transformed feature operations.
    """
    outputs = {}

    # Scale these features to the range [0,1]
    for key in NUMERIC_FEATURE_KEYS:
        outputs[transformed_name(key)] = tft.scale_to_0_1(
            inputs[key])
    
    # Convert strings to indices in a vocabulary
    for key in CATEGORICAL_FEATURE_KEYS:
        outputs[transformed_name(key)] = tft.compute_and_apply_vocabulary(inputs[key])

    # Convert the label strings to an index
    outputs[transformed_name(LABEL_KEY)] = tft.compute_and_apply_vocabulary(inputs[LABEL_KEY])

    return outputs

In [13]:
# Ignore the warnings
tf.get_logger().setLevel('ERROR')

# a temporary directory is needed when analyzing the data
# with tft_beam.Context(temp_dir=tempfile.mkdtemp()):
    
#     # define the pipeline using Apache Beam syntax
#     transformed_dataset, transform_fn = (
        
#         # analyze and transform the dataset using the preprocessing function
#         (train_df, raw_data_metadata) | tft_beam.AnalyzeAndTransformDataset(
#             preprocessing_fn)
#     )

transform_fn = tft_beam.AnalyzeDataset(preprocessing_fn).expand( (train_df, raw_data_metadata))
# transformed_dataset = tft_beam.TransformDataset().expand((train_df, transform_fn))

# # unpack the transformed dataset
# transformed_data, transformed_metadata = transformed_dataset

# # print the results
# print('\nRaw data:\n{}\n'.format(pprint.pformat(train_df)))
# print('Transformed data:\n{}'.format(pprint.pformat(transformed_data)))
transform_fn

usage: ipykernel_launcher.py [-h] [--dataflow_endpoint DATAFLOW_ENDPOINT]
                             [--project PROJECT] [--job_name JOB_NAME]
                             [--staging_location STAGING_LOCATION]
                             [--temp_location TEMP_LOCATION] [--region REGION]
                             [--service_account_email SERVICE_ACCOUNT_EMAIL]
                             [--no_auth]
                             [--template_location TEMPLATE_LOCATION]
                             [--label LABELS] [--update]
                             [--transform_name_mapping TRANSFORM_NAME_MAPPING]
                             [--enable_streaming_engine]
                             [--dataflow_kms_key DATAFLOW_KMS_KEY]
                             [--create_from_snapshot CREATE_FROM_SNAPSHOT]
                             [--flexrs_goal {COST_OPTIMIZED,SPEED_OPTIMIZED}]
                             [--dataflow_service_option DATAFLOW_SERVICE_OPTIONS]
                           

Traceback (most recent call last):
  File "/usr/lib/python3.10/argparse.py", line 1878, in parse_known_args
    namespace, args = self._parse_known_args(args, namespace)
  File "/usr/lib/python3.10/argparse.py", line 2091, in _parse_known_args
    start_index = consume_optional(start_index)
  File "/usr/lib/python3.10/argparse.py", line 2031, in consume_optional
    take_action(action, args, option_string)
  File "/usr/lib/python3.10/argparse.py", line 1939, in take_action
    argument_values = self._get_values(action, argument_strings)
  File "/usr/lib/python3.10/argparse.py", line 2475, in _get_values
    self._check_value(action, value)
  File "/usr/lib/python3.10/argparse.py", line 2531, in _check_value
    raise ArgumentError(action, msg % args)
argparse.ArgumentError: argument --flexrs_goal: invalid choice: '/root/.local/share/jupyter/runtime/kernel-v2-734dwkr8S9Q4btA.json' (choose from 'COST_OPTIMIZED', 'SPEED_OPTIMIZED')

During handling of the above exception, another exceptio

TypeError: object of type 'NoneType' has no len()